In [1]:
#combine conventional DFD and information weights to produce information-weighted optical flow images
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
import math
import glob
import os 

In [2]:
#folder and image set up
start = time.time()
#
imagePrefix='DFDEc_gimbal0_'
imageSuffix='.png'
pathImage='../scout_video_09/'
filePrefix='gimbal0_'
fileSuffix='.jpg'
#
tileSpan=15
dimX=1920
dimY=1080
searchRange=25
searchStep=1
windowSpanX=20
windowSpanY=20
windowShiftX=30
windowShiftY=30
#compensation parameters
speedY=5 #m/s
frameRate=10 #Hz
pixelSize=0.1 #m
compensateY= speedY / frameRate / pixelSize

#DFD set up
DFDheight=1030
DFDwidth=1870

#Rip current analysis set up
ripNoImages=50
ripNoRowWindow=10 #windowSpanY and windowShiftY
oceanDepthShallow=0.1
oceanDepthDeep=1
ripIdxStart=2440
ripIdxEnd=3150

In [3]:
# matrix set up
svSx=int((DFDwidth-windowSpanX*2)/windowShiftX)
svSy=int((DFDheight-windowSpanY*2)/windowShiftY)
#
oceanDepthSlice=np.arange(oceanDepthShallow, oceanDepthDeep, (oceanDepthDeep-oceanDepthShallow)/(svSx+1))
oceanDepthMatrix=np.repeat([oceanDepthSlice], ripNoRowWindow, axis=0)
#quadratic ocean risk model based rip current flow
oceanRiskSlice=np.multiply(oceanDepthSlice, oceanDepthSlice)
oceanRiskMatrix=np.multiply(oceanDepthMatrix, oceanDepthMatrix)

In [4]:
#
countDFDx=np.zeros(DFDheight)
countDFDx.shape
print(ripIdxEnd)

3150


In [5]:
#combine DFD and information density and save
start = time.time()
for n in range(ripIdxStart, ripIdxEnd):
    #Initialize Averaged DFD vector matrix
    vectorSampleX=np.zeros((svSy+1, svSx+1))
    #vectorSampleY=np.zeros((svSy+1, svSx+1))
    ripDFDx=np.zeros((DFDheight, DFDwidth))
    countDFDx=np.zeros((DFDheight,1))
    DFDx=np.load(pathImage + filePrefix +"DFDx_" + str(n) + ".npy")
    DFDy=np.load(pathImage + filePrefix +"DFDy_" + str(n) + ".npy")
    DFDEc=np.load(pathImage + filePrefix +"DFDEc_" + str(n) + ".npy")

    f=plt.figure(dpi=310)
    f.set_figwidth(8)
    f.set_figheight(12)

    filename=filePrefix + str(n) + fileSuffix
    img_ref = mpimg.imread(pathImage + filename) / 256
    imgplot_1 = plt.imshow(img_ref)

    #draw average DFD vectors
    for x in range(windowSpanX, DFDwidth-windowSpanX+1, windowShiftX ):
        for y in range(windowSpanY, DFDheight-windowSpanY+1, windowShiftY):
            vX=np.mean(DFDx[y-windowSpanY:y+windowSpanY,x-windowSpanX:x+windowSpanX])
            vY=np.mean(DFDy[y-windowSpanY:y+windowSpanY,x-windowSpanX:x+windowSpanX])
            vC=np.mean(DFDEc[y-windowSpanY:y+windowSpanY,x-windowSpanX:x+windowSpanX])
            vX0=x + searchRange + tileSpan
            vY0=y + searchRange + tileSpan
            #print(vX0, vX0+vX , vY0, vY0+vY)
            if vX>0:
                lineColor='r'
            else:
                lineColor='b'
            vX1=vX0+4*vC*vX
            vY1=vY0+4*vC*(vY-compensateY)
            if vX1<0:
                vX1=0
            if vY1<0:
                vY1=0
            if vX1>dimX-1:
                vX1=dimX-1
            if vY1>dimY-1:
                vY1=dimY-1
            plt.plot([vX0, vX1], [vY0, vY1], lineColor)
    
    plt.axis('off')
    filename=imagePrefix + str(n) + imageSuffix    
    plt.savefig(pathImage + filename, bbox_inches = 'tight', pad_inches = 0)
    plt.close()
    #measure time
    end = time.time()
    print(n, end - start)

2440 1.8015635013580322
2441 3.55474591255188
2442 5.262216806411743
2443 6.931685447692871
2444 8.61565375328064
2445 10.5194993019104
2446 12.116346597671509
2447 14.624977827072144
2448 16.186116933822632
2449 17.874577045440674
2450 19.357929706573486
2451 20.846614599227905
2452 22.574034214019775
2453 24.056925535202026
2454 25.53992795944214
2455 27.301205158233643
2456 28.78449535369873
2457 30.277395486831665
2458 31.75773787498474
2459 33.576170682907104
2460 35.04907274246216
2461 36.549763679504395
2462 38.04407477378845
2463 39.53006410598755
2464 41.47976994514465
2465 43.00107192993164
2466 44.501099824905396
2467 46.00517177581787
2468 47.50205397605896
2469 49.01019740104675
2470 51.00208830833435
2471 52.495643615722656
2472 53.98523569107056
2473 55.46290874481201
2474 57.02603816986084
2475 58.48718476295471
2476 59.95223665237427
2477 61.410423040390015
2478 63.60679841041565
2479 65.11428761482239
2480 66.65297245979309
2481 68.24994111061096
2482 69.7968530654907

EOFError: No data left in file